In [1]:
import pandas as pd

In [2]:
import sys
sys.path.append('/home/ubuntu/JAI/')

In [3]:
import pickle
from pathlib import Path
import datetime as dt
import logging
import numpy as np
import pandas as pd
import spacy
from en_ner_guardian.trf_tensor_to_vec import *

/home/ubuntu/.conda/envs/jai_new_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
spacy.prefer_gpu()

True

In [5]:
model = 'gu_trf'
nlp = spacy.load('/home/ubuntu/JAI/en_ner_guardian/en_ner_guardian/en_ner_guardian/en_ner_guardian-1.0.3/')
nlp.add_pipe('tensor2attr')

/home/ubuntu/.conda/envs/jai_new_venv/lib/python3.8/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_ner_guardian' (1.0.3) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/ubuntu/.conda/envs/jai_new_venv/lib/python3.8/site-packages/spacy_transformers/pipeline_component.py:405: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [7]:
gu_data = pd.read_csv('gu_resampled_by_section_id_full_name_all_aliases_kb_match_paragraphs.csv',index_col=0)

In [8]:
gu_data.shape

(152616, 5)

In [9]:
gu_data=gu_data[['paragraphs']]

In [10]:
gu_data=gu_data.drop_duplicates()

In [15]:
gu_data=gu_data.sample(20000, random_state=14)

In [16]:
doc = nlp('Donald Trump is a president.')

In [22]:
str(doc.ents[0])

'Donald Trump'

In [75]:
paragraph_dict={}
ner_lst=[]
for index, paragraph in gu_data.iterrows():
    paragraph_dict[index] = paragraph[0]
    ent_lst=[]
    doc = nlp(paragraph[0])
    for ent in doc.ents:
        ent_lst.append([index,
                        str(ent),
                        ent.start_char,
                        ent.end_char
                      ])
    ner_lst.extend(ent_lst)

In [80]:
paragraph_ner = pd.DataFrame(ner_lst, columns=['index','ent','start_char','end_char'])

In [81]:
gu_paragraphs = pd.DataFrame.from_dict(paragraph_dict, orient='index', columns=['paragraph']).reset_index()

In [85]:
paragraph_ner = paragraph_ner.merge(gu_paragraphs)

In [87]:
paragraph_ner

,index,ent,start_char,end_char,paragraph
0,2453,Putin,6,11,Other Putin critics have died in the UK in mys...
1,2453,Nikolai Glushkov,128,144,Other Putin critics have died in the UK in mys...
2,2453,Boris Berezovsky,341,357,Other Putin critics have died in the UK in mys...
3,41366,Jackson,51,58,Even as senators focused on the historic natur...
4,41366,Mazie Hirono,301,313,Even as senators focused on the historic natur...
...,...,...,...,...,...
39122,544427,Nancy Pelosi,19,31,"The House speaker, Nancy Pelosi, ordered flags..."
39123,185048,Narendra Modi,120,133,The Bharatiya Janata party (BJP) has repeatedl...
39124,55487,Starmer,103,110,Labour’s progress on Thursday has been oversha...
39125,55487,Johnson,204,211,Labour’s progress on Thursday has been oversha...


In [94]:
paragraph_ner.to_csv('20k_sample_paragraphs_ner.csv')

In [88]:
dataset='full'# OR'open_sanctions'# OR 'lilsis'
kb_iteration='_2022_11_07'

In [92]:
kb_data=pd.read_csv(f'../kb_datasets/kb_entities_{dataset}{kb_iteration}.csv',index_col=0)

/tmp/ipykernel_8742/1633190115.py:1: DtypeWarning: Columns (2,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  kb_data=pd.read_csv(f'../kb_datasets/kb_entities_{dataset}{kb_iteration}.csv',index_col=0)


In [98]:
paragraph_ner['single_name']=paragraph_ner['ent'].apply(lambda x: x if len(x.split())==1 else x.split()[-1])

In [101]:
kb_data['single_name'] = kb_data['name'].apply(lambda x: x if len(x.split())==1 else x.split()[-1])

In [102]:
kb_data[['id','name','single_name','desc']]

,id,name,single_name,desc
0,acf-00040861bc3f593000830d987d09967ef3503ef1,Kolyvanov Egor,Egor,Kolyvanov Egor is a Russian propagandist: host...
1,acf-0011c68a768924609dc5da5707ac7fa4c4d645a2,Shipov Sergei Yurievich,Yurievich,Shipov Sergei Yurievich is a Russian chess pla...
2,acf-001e7e4c0363f08f1e784c230457960b84a6416f,Egorov Ivan Mikhailovich,Mikhailovich,Egorov Ivan Mikhailovich is a Deputy of the St...
3,acf-002c208139012c8d93b6298358188d7cadafe648,Goreslavsky Alexey Sergeyevich,Sergeyevich,Goreslavsky Alexey Sergeyevich is a Russian jo...
4,acf-002cc8fdf8fe41185091a7cb6c598663e7a22eb5,Samoilova Natalya Vladimirovna,Vladimirovna,Samoilova Natalya Vladimirovna is a Russian si...
...,...,...,...,...
428514,413772,Cory Bernardi,Bernardi,Cory Bernardi is a Australian politician and r...
428515,Q47668202,Jalbasürengiin Batzandan,Batzandan,Jalbasürengiin Batzandan is a Mongolian politi...
428516,13488,Patrick Murphy,Murphy,Patrick Murphy is a former US Representative f...
428517,Q28033808,Sharif Street,Street,Sharif Street is a American politician from Pe...


In [104]:
paragraph_ner = paragraph_ner.merge(kb_data[['id','name','single_name','desc']])

In [111]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_ = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [118]:
paragraph_ner['paragraph'] = paragraph_ner['paragraph'].iloc[:10].apply(lambda x: set([w for w in x.split() if w not in stopwords_]))

In [116]:
paragraph_ner

,index,ent,start_char,end_char,paragraph,single_name,id,name,desc
0,2453,Putin,6,11,Other Putin critics have died in the UK in mys...,Putin,Q15087614,Roman Igorevich Putin,Roman Igorevich Putin is a Роман Путін бере уч...
1,2453,Putin,6,11,Other Putin critics have died in the UK in mys...,Putin,Q1833348,Igor Alexandrovich Putin,Igor Alexandrovich Putin is a Russian politici...
2,2453,Putin,6,11,Other Putin critics have died in the UK in mys...,Putin,Q27303716,Alexander Spiridonovich Putin,This person is called Alexander Spiridonovich ...
3,2453,Putin,6,11,Other Putin critics have died in the UK in mys...,Putin,Q7747,Vladimir Vladimirovitj Putin,Vladimir Vladimirovitj Putin is a President of...
4,2453,Putin,6,11,Other Putin critics have died in the UK in mys...,Putin,149070,Vladimir Putin,Vladimir Putin is a President of Russia. This...
...,...,...,...,...,...,...,...,...,...
3904031,709250,Abdel Mahdi,0,11,Abdel Mahdi congratulated his successor and th...,Mahdi,Q4670492,Abubakar Mahdi,Abubakar Mahdi is a Nigerian politician. This ...
3904032,709250,Abdel Mahdi,0,11,Abdel Mahdi congratulated his successor and th...,Mahdi,Q6930870,Mubarak Al Fadil Al Mahdi,Mubarak Al Fadil Al Mahdi is a Sudanese politi...
3904033,709250,Abdel Mahdi,0,11,Abdel Mahdi congratulated his successor and th...,Mahdi,Q75807853,Sammy Mahdi,Sammy Mahdi is a Belgian politician. This pers...
3904034,709250,Abdel Mahdi,0,11,Abdel Mahdi congratulated his successor and th...,Mahdi,Q95626382,Adil Abdallah Mahdi,Adil Abdallah Mahdi is a Ba'th party regional ...
